In [5]:
import tensorflow as tf # 1.15 ver.

node1 = tf.constant(10, dtype=tf.float32) # 상수 node를 만들자
node2 = tf.constant(30, dtype=tf.float32)

node3 = node1 + node2

sess = tf.Session() # 그래프를 실행시켜주는 Session이 필요하다

print(sess.run(node1)) # graph를 실행시킴
print(sess.run(node3))
print(sess.run([node1, node3]))

10.0
40.0
[10.0, 40.0]


In [8]:
import tensorflow as tf

# 만약 다차원 데이터가 입력되려면 shape을 지정해야함. 나중에 입력 받을 값을 실수로 사용하겠다
# node1 = tf.placeholder(shape=[2,2], dtype=tf.float32)

node1 = tf.placeholder(dtype=tf.float32) # scalar
node2 = tf.placeholder(dtype=tf.float32)

node3 = node1 + node2

sess = tf.Session()

result = sess.run(node3, feed_dict={node1: 10, node2: 30}) # 입력이 없는 상태에서 sess을 실행하면 오류. feed로 초기값을 지정해줌

print(result)

40.0


In [24]:
# Q1, Q2, Q3, Exam 예제에 Tensorflow를 사용해서 다중선형회귀를 구현해보자
import numpy as np
import pandas as pd
import tensorflow as tf

# Raw Data Loading
df = pd.read_csv('./data/student_exam_score.csv')
# display(df.head())

# Training Data Set
x_data = df.drop('exam', axis=1, inplace=False) # (25, 3)
t_data = df['exam'].values.reshape(-1,1) # (25, 1)

# 학습종료 후 예측
# predict_data = np.array([[90, 100, 95]]) # (1, 3)

# Placeholder
X = tf.placeholder(shape=[None, 3], dtype=tf.float32) # Shape는 입력값이 어떻든 상관없지만, 열은 3개다
T = tf.placeholder(shape=[None, 1], dtype=tf.float32) # Shape는 입력값이 어떻든 상관없지만, 열은 1개다

# Weight, bias. 파이썬의 변수처럼 텐서플로에도 변수처럼 작동하는 Node를 쓰자
W = tf.Variable(tf.random.normal([3,1])) # Shape는 (3, 1). 표준 정규분포에서 난수를 추출
b = tf.Variable(tf.random.normal([1]))

# model(hypothesis, 가설) y = Wx + b
H = tf.matmul(X,W) + b

# loss function. Error(오차, t-y)를 가지고 동작하는 Node
loss = tf.reduce_mean(tf.square(H - T)) # square과 reduce_mean으로 오차의 제곱의 평균(MSE) 구하자

# train node를 생성하자. 경사하강법과 learning rate와 최소 MSE. loss를 가지고 동작하는 Node
train = tf.train.GradientDescentOptimizer(learning_rate=1e-7).minimize(loss)

# Session을 생성하고 초기화를 진행
sess = tf.Session()
sess.run(tf.global_variables_initializer()) # 그래프를 실행하기 전에 초기화 구문이 선행되어야 함

# 반복 학습
for step in range(300000):
    _, W_val, b_val, loss_val = sess.run([train, W, b, loss], feed_dict={X: x_data, T: t_data})
    if step % 30000 == 0:
        print('W : {}, b: {}, loss : {}'.format(W_val, b_val, loss_val))

W : [[ 0.35620126]
 [ 0.6802199 ]
 [-1.2736073 ]], b: [1.7118871], loss : 34067.4140625
W : [[ 1.0466127 ]
 [ 1.3010069 ]
 [-0.31793556]], b: [1.7212033], loss : 70.16020965576172
W : [[ 0.99326324]
 [ 1.1868298 ]
 [-0.15451476]], b: [1.7212033], loss : 55.9371337890625
W : [[ 0.9441845 ]
 [ 1.0882058 ]
 [-0.01044486]], b: [1.7212033], loss : 44.94955062866211
W : [[0.8990436 ]
 [1.0030878 ]
 [0.11662561]], b: [1.7212033], loss : 36.45235061645508
W : [[0.85749674]
 [0.929632  ]
 [0.22878143]], b: [1.7212033], loss : 29.87066078186035
W : [[0.81922096]
 [0.8663443 ]
 [0.32781464]], b: [1.7212033], loss : 24.767221450805664
W : [[0.78408545]
 [0.8118457 ]
 [0.41522998]], b: [1.7212033], loss : 20.807594299316406
W : [[0.7515445 ]
 [0.765078  ]
 [0.49254933]], b: [1.7212033], loss : 17.726661682128906
W : [[0.72171825]
 [0.72483176]
 [0.5608492 ]], b: [1.7212033], loss : 15.328995704650879


In [25]:
# 학습종료 후 예측
result = sess.run(H, feed_dict={X: np.array([[89, 100, 95]])})
print(result) # 30만번 : [[187.72133]], [[195.62724]], [[191.56349]] / 40만번 : [[194.6791]]

[[191.56349]]
